<a href="https://colab.research.google.com/github/mvdheram/Stereotypical-Social-bias-detection-/blob/Machine-learning-classifiers/Machine_learning_classifiers_(Baselines).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Final selection :

* SVM Classifier using selected features 
  * Reference : Linguistic models for detecting bias https://aclanthology.org/P13-1162.pdf
  * 

  